In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim


from models.learned_qp_solver_bi_cycle_vel_part import MLP_Init, learned_qp_solver
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
# torch.set_default_dtype(torch.float64)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:


num = 50
t = 0.1


vel_min = 0.0
vel_max = 30.0

acc_min = -6.0
acc_max = 6.0

jerk_min = -6.0
jerk_max = 6.0



In [4]:
#### creating dataset 

dataset_size = 200000

vel_samples = np.random.uniform(vel_min, vel_max,  (dataset_size, num)    )
# vel_samples = np.random.multivariate_normal(np.zeros(num), vel_max*np.identity(num), (dataset_size, ) )

vel_samples = np.clip( vel_samples, vel_min, vel_max  )



vel_init = np.random.uniform(vel_min, vel_max, (dataset_size, 1)  )


inp = np.hstack(( vel_samples,  vel_init ))

inp_min = inp.min()
inp_max = inp.max()

# normalization_const = (vel_max-vel_min)

inp_mean, inp_std = inp.mean(), inp.std()

median_ = np.median(inp, axis=0)
q1 = np.percentile(inp, 25, axis=0)
q3 = np.percentile(inp, 75, axis=0)
iqr_ = q3 - q1


# Handle constant features
iqr_ = np.where(iqr_ == 0, 1, iqr_)

print(np.array_repr(median_))
print(np.array_repr(iqr_))


# inp_mean = inp_min 
# inp_std = inp_max


median_ = torch.tensor(median_).float().to(device)
iqr_ = torch.tensor(iqr_).float().to(device)



print(inp_mean, inp_std)


array([15.00359767, 14.99914977, 14.96252152, 14.99510703, 15.01073102,
       14.91806615, 15.04748382, 14.97385411, 14.91958671, 14.96674753,
       14.99380754, 15.04231799, 15.00645121, 14.98914404, 15.00864676,
       14.94916843, 15.06330505, 14.93835699, 14.98932302, 15.01801051,
       14.9965207 , 15.04619562, 14.94372447, 14.98282104, 14.98978685,
       15.00803807, 14.99560976, 15.01813398, 15.00355276, 14.98718907,
       14.96198765, 14.95042323, 15.01404593, 15.00263588, 14.96536441,
       15.0139652 , 14.9962657 , 14.99809796, 14.98118898, 14.99698807,
       15.01943598, 15.04303794, 14.92765476, 14.99997739, 14.96185262,
       14.9847094 , 14.99657718, 14.96941295, 15.00560244, 14.99084167,
       14.98383471])
array([15.00926073, 15.00732423, 14.98583147, 14.98943907, 14.94082761,
       15.00961344, 15.0142066 , 15.02197942, 15.02750788, 15.01522525,
       14.99040227, 14.99389029, 15.0369318 , 15.00210689, 15.02226131,
       14.92891145, 15.01925311, 14.9965225

In [5]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, vel_init, vel_samples):
		
		# State Data
		self.inp = inp
  
		self.vel_init = vel_init
		
		# Expert Coeff
		self.vel_samples = vel_samples

		
		
	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		inp = self.inp[idx]
		vel_init = self.vel_init[idx]

		
		# Outputs
		vel_samples = self.vel_samples[idx]
	
		
		
		return torch.tensor(inp).float(), torch.tensor(vel_init).float(), torch.tensor(vel_samples).float()
	

# Batch Size - 3k or 4k
batch_size = 2056

# pcd_data = pcd_data.reshape(data_set_size, 2, 200)

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, vel_init,  vel_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)



In [6]:
# Differentiable Layer
num_batch = train_loader.batch_size


mlp_init_inp_dim = np.shape(inp)[1]
mlp_init_hidden_dim = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim = 2*num+(2*num+2*num+2*(num-1)+2*(num-2))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init = MLP_Init(mlp_init_inp_dim, mlp_init_hidden_dim, mlp_init_out_dim  )


model = learned_qp_solver(num_batch, num, t, mlp_init, inp_mean, inp_std).to(device)
# model.load_state_dict(torch.load('./weights/learned_qp_bi_cycle_vel_part.pth'))



In [7]:
epochs = 800
step = 0 
beta = 1.0 # 3.5

optimizer = optim.AdamW(model.parameters(), lr = 1e-4, weight_decay=6e-5)
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3, weight_decay=6e-5)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)

avg_train_loss, avg_loss_primal, avg_fixed_point_loss, avg_proj_loss = [], [], [], []
model.train()
for epoch in range(epochs):
	
	# Train Loop
	losses_train, aug_losses_primal, aug_losses_fixed_point, aug_losses_proj = [], [], [], []
	
	for (inp, vel_init, vel_samples) in train_loader:
		
		# Input and Output 
  
		################################################################################################

		# print(goal_des[0])
		# print(y_lb[0], y_ub[0])

  
		inp = inp.to(device)
		vel_init = vel_init.to(device)
		
		vel_samples = vel_samples.to(device)
	
		
		vel_projected, res_primal_stack, res_fixed_point_stack, accumulated_res_primal, accumulated_res_fixed_point = model(inp, vel_init, vel_samples, vel_max, vel_min, acc_max, acc_min, jerk_max, jerk_min, inp_min, inp_max, median_, iqr_)
		loss, primal_loss, fixed_point_loss, proj_loss = model.ss_loss(accumulated_res_primal, accumulated_res_fixed_point, vel_projected, vel_samples)
	
		optimizer.zero_grad()
		loss.backward()
		# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
		optimizer.step()
		
		losses_train.append(loss.detach().cpu().numpy()) 
		aug_losses_primal.append(primal_loss.detach().cpu().numpy())
		aug_losses_fixed_point.append(fixed_point_loss.detach().cpu().numpy())
		aug_losses_proj.append(proj_loss.detach().cpu().numpy())
		# aug_losses_steer.append(steer_loss.detach().cpu().numpy())

		
	# scale = scale*1.2	
		
	if epoch % 4 == 0:    
		print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, aug_loss_primal: {np.average(aug_losses_primal):.3f}, aug_loss_fixed_point: {np.average(aug_losses_fixed_point):.3f}, aug_loss_proj: {np.average(aug_losses_proj):.3f} ")

	step += 0.15 #0.15
	# scheduler.step()
	avg_train_loss.append(np.average(losses_train)), avg_loss_primal.append(np.average(aug_losses_primal)), avg_fixed_point_loss.append(np.average(aug_losses_fixed_point)),  avg_proj_loss.append(np.average(aug_losses_proj))
	
		



Epoch: 1, Train Loss: 59.988, aug_loss_primal: 0.673, aug_loss_fixed_point: 18.836, aug_loss_proj: 40.480 
Epoch: 5, Train Loss: 44.756, aug_loss_primal: 0.148, aug_loss_fixed_point: 2.991, aug_loss_proj: 41.617 
Epoch: 9, Train Loss: 43.399, aug_loss_primal: 0.104, aug_loss_fixed_point: 1.750, aug_loss_proj: 41.546 
Epoch: 13, Train Loss: 42.893, aug_loss_primal: 0.078, aug_loss_fixed_point: 1.367, aug_loss_proj: 41.448 
Epoch: 17, Train Loss: 42.310, aug_loss_primal: 0.046, aug_loss_fixed_point: 1.052, aug_loss_proj: 41.212 
Epoch: 21, Train Loss: 41.833, aug_loss_primal: 0.027, aug_loss_fixed_point: 0.781, aug_loss_proj: 41.025 
Epoch: 25, Train Loss: 41.610, aug_loss_primal: 0.020, aug_loss_fixed_point: 0.643, aug_loss_proj: 40.947 
Epoch: 29, Train Loss: 41.484, aug_loss_primal: 0.017, aug_loss_fixed_point: 0.572, aug_loss_proj: 40.894 
Epoch: 33, Train Loss: 41.394, aug_loss_primal: 0.015, aug_loss_fixed_point: 0.525, aug_loss_proj: 40.854 
Epoch: 37, Train Loss: 41.322, aug_loss

KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), './weights/learned_qp_bi_cycle_vel_part.pth')